In [1]:
# Import the usual packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data as pandas DataFrames
order = pd.read_csv("order.csv")
constraint = pd.read_csv("constraints.csv")

In [5]:
# Calculate the set amount from origin port and set demand for destination
summary = order.groupby(['Product ID', 'Origin Port'])[['Total Weight']].sum()
demand = order.groupby(['Product ID', 'Destination Port'])[['Total Weight']].sum()

# Assign variables of set amounts from origin port
MIA1 = summary.iloc[0, 0]
SF1 = summary.iloc[1, 0]
MIA2 = summary.iloc[2, 0]
SF2 = summary.iloc[3, 0]
P1 = MIA1 + SF1
P2 = MIA2 + SF2
ef = P1 + P2

# Assign variables of set demand for destination
O1 = demand.iloc[0, 0]
B1 = demand.iloc[1, 0]
H1 = demand.iloc[2, 0]
R1 = demand.iloc[3, 0]
O2 = demand.iloc[4, 0]
B2 = demand.iloc[5, 0]
H2 = demand.iloc[6, 0]
R2 = demand.iloc[7, 0]

In [2]:
# Create a blank model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-25


In [9]:
# Add variables
SM1 = m.addVar(lb = MIA1, ub = MIA1)
SM2 = m.addVar(lb = MIA2, ub = MIA2)
SS1 = m.addVar(lb = SF1, ub = SF1)
SS2 = m.addVar(lb = SF2, ub = SF2)
M1B1 = m.addVar(lb = 0, ub = P1)
M2B2 = m.addVar(lb = 0, ub = P2)
M1H1 = m.addVar(lb = 0, ub = P1)
M2H2 = m.addVar(lb = 0, ub = P2)
M1R1 = m.addVar(lb = 0, ub = P1)
M2R2 = m.addVar(lb = 0, ub = P2)
M1O1 = m.addVar(lb = 0, ub = P1)
M2O2 = m.addVar(lb = 0, ub = P2)
S1B1 = m.addVar(lb = 0, ub = P1)
S2B2 = m.addVar(lb = 0, ub = P2)
S1H1 = m.addVar(lb = 0, ub = P1)
S2H2 = m.addVar(lb = 0, ub = P2)
S1R1 = m.addVar(lb = 0, ub = P1)
S2R2 = m.addVar(lb = 0, ub = P2)
S1O1 = m.addVar(lb = 0, ub = P1)
S2O2 = m.addVar(lb = 0, ub = P2)
B1H1 = m.addVar(lb = 0, ub = P1)
B2H2 = m.addVar(lb = 0, ub = P2)
B1O1 = m.addVar(lb = 0, ub = P1)
B2O2 = m.addVar(lb = 0, ub = P2)
B1T = m.addVar(lb = B1, ub = B1)
B2T = m.addVar(lb = B2, ub = B2)
H1T = m.addVar(lb = H1, ub = H1)
H2T = m.addVar(lb = H2, ub = H2)
R1H1 = m.addVar(lb = 0, ub = P1)
R2H2 = m.addVar(lb = 0, ub = P2)
R101 = m.addVar(lb = 0, ub = P1)
R202 = m.addVar(lb = 0, ub = P2)
R1T = m.addVar(lb = R1, ub = R1)
R2T = m.addVar(lb = R2, ub = R2)
O1T = m.addVar(lb = O1, ub = O1)
O2T = m.addVar(lb = O2, ub = O2) # and more.....

In [10]:
# Model creation
E = 36 # Number of edges
costs = constraint["Cost"]
list_var = [SM1,SM2,SS1,SS2,M1B1,M2B2,M1H1,M2H2,M1R1,M2R2,M1O1,M2O2,S1B1,S2B2,
            S1H1,S2H2,S1R1,S2R2,S1O1,S2O2,B1H1,B2H2,B1O1,B2O2,B1T,B2T,H1T,H2T,
            R1H1,R2H2,R101,R202,R1T,R2T,O1T,O2T]
var = pd.Series(list_var)

# Set objective
m.setObjective(gp.quicksum(costs[i]*var[i] for i in range(E)), GRB.MINIMIZE)

# Add constraints: flow in = flow out
m.addConstr(ef-(SM1+SM2+SS1+SS2)==0) # Sink
m.addConstr(SM1-(M1B1+M1H1+M1R1+M1O1)==0) # M1
m.addConstr(SM2-(M2B2+M2H2+M2R2+M2O2)==0) # M2
m.addConstr(SS1-(S1B1+S1H1+S1R1+S1O1)==0) # S1
m.addConstr(SS2-(S2B2+S2H2+S2R2+S2O2)==0) # S2
m.addConstr((M1B1+S1B1)-(B1H1+B1O1+B1T)==0) # B1
m.addConstr((M2B2+S2B2)-(B2H2+B2O2+B2T)==0) # B2
m.addConstr((M1H1+S1H1+B1H1+R1H1)-H1T==0) # H1
m.addConstr((M2H2+S2H2+B2H2+R2H2)-H2T==0) # H2
m.addConstr((M1R1+S1R1)-(R1H1+R101+R1T)==0) # R1
m.addConstr((M2R2+S2R2)-(R2H2+R202+R2T)==0) # R2
m.addConstr((M1O1+S1O1+B1O1+R101)-O1T==0) # O1
m.addConstr((M2O2+S2O2+B2O2+R202)-O2T==0) # O2
m.addConstr((B1T+B2T+H1T+H2T+R1T+R2T+O1T+O2T)-ef==0) # T

<gurobi.Constr *Awaiting Model Update*>

In [12]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 4800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 14 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x143e95ad
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 7e-01]
  Bounds range     [5e+07, 3e+08]
  RHS range        [5e+08, 5e+08]
Presolve removed 2 rows and 12 columns
Presolve time: 0.01s
Presolved: 12 rows, 24 columns, 48 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6862007e+08   1.051676e+09   0.000000e+00      0s
      13    3.0453116e+08   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.045311554e+08


In [17]:
var = constraint["Label"]
print("Minimum total cost: ", m.ObjVal, "\n")
print("Optimal shipping route:")
for i in range(E):
  print(var[i], ":", list_var[i].X)

Minimum total cost:  304531155.4050001 

Optimal shipping route:
SM1 : 110866057.5
SM2 : 142444854.3
SS1 : 119927600.5
SS2 : 152599434.4
M1B1 : 59130376.5
M2B2 : 68987598.40000002
M1H1 : 0.0
M2H2 : 0.0
M1R1 : 47737074.0
M2R2 : 73457255.89999999
M1O1 : 3998607.0
M2O2 : 0.0
S1B1 : 0.0
S2B2 : 0.0
S1H1 : 54524988.0
S2H2 : 74001274.9
S1R1 : 0.0
S2R2 : 2001434.400000006
S1O1 : 65402612.5
S2O2 : 76596725.1
B1H1 : 0.0
B2H2 : 0.0
B1O1 : 0.0
B2O2 : 0.0
B1T : 59130376.5
B2T : 68987598.4
H1T : 54524988.0
H2T : 74001274.9
R1H1 : 0.0
R2H2 : 0.0
R1O1 : 0.0
R2O2 : 0.0
R1T : 47737074.0
R2T : 75458690.3
O1T : 69401219.5
O2T : 76596725.1
